# Config

In [11]:
from plotly.offline import init_notebook_mode, iplot # from plotly.offline import init_notebook_mode, iplot # 
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)

In [12]:
import numpy as np
from scipy.integrate import odeint

# Simplest Plot

In [13]:
x = list(range(0,10))
y = [c*c for c in x]
# iplot([{"x": x, "y": y}])

# Dynamics (Equations of Motion)

$m\ddot{x} + b \dot{x} + kx = u$

$s = \begin{bmatrix}x \\ \dot{x}\end{bmatrix}$

In [14]:
# Equations of motion
def get_x_dotdot(x, x_dot, u):
    m = 10
    b = 1*0
    k = 1
    x_dotdot = (u-k*x-b*x_dot)/m
    return x_dotdot

# ODE
def get_s_dot(y,t,u):
    x, x_dot = y
    x_dotdot = get_x_dotdot(x,x_dot, u)
    s_dot = [x_dot, x_dotdot]
    return s_dot
    
# PD controller    
def get_u(x,x_dot):
    kp = 10
    kd = 6
    u = -kp*x-kd*x_dot
    return u    

# Simulation

Dynamics in continous time; controller in discrete time (zero order hold)

In [65]:
# Initial Conditions
s0 = [3, 0]

# Initialize empty result vectors
t_all = np.empty(shape=[0, 1]) 
s_all = np.empty(shape=[0, 2]) 
end = -1

# Time settings
dt    = 0.5 # Controller sampling time
t_end = 10  # Simulation time
Tdis = np.arange(0,t_end,dt) # Discrete time samples

# At every sample do
for tnow in Tdis:
    
    # Given the current state, compute a control signal
    x,x_dot = s0
    u = get_u(x,x_dot)
    
    # Integrate equation of motion until next sample while holding control
    simsteps_per_sample = 10
    t = np.linspace(tnow,tnow+dt,simsteps_per_sample+1)
    s = odeint(get_s_dot, s0, t, args=(u,))
    
    # The final y state is the initial state for next time
    s0 = s[end,:]
    
    # Append the results, except the final state. That will be appended as the initial state of the next one
    t_all = np.append(t_all,t[0:end])
    s_all = np.append(s_all,s[0:end,:], axis = 0)
    
# Match time and output signals for plotting    
pos_plot = Scatter(x = t_all, y = s_all[:, 0], name = 'position')   
vel_plot = Scatter(x = t_all, y = s_all[:, 1], name = 'speed')   
    
# Plot results inline    
iplot([pos_plot,vel_plot])   

In [67]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
init_notebook_mode(connected=True)


coordinate_pair_x = [[0,0]          for i, t in enumerate(t_all)]
coordinate_pair_y = [[0,s_all[i,0]] for i, t in enumerate(t_all)]

# The actual frames
framedata = [ {'data': [{'x': coordinate_pair_x[i], 'y': coordinate_pair_y[i]}]}  for i, t in enumerate(t_all)]

# Empty data to display nothing at first. To work around a plotly problem where you can't choose the framerate for the first run.
dummydata = [ {'data': [{'x': [], 'y': []}]}]

figure = {'data': [{'x': [], 'y': []}],
          'layout': {'xaxis': {'range': [-1, 1], 'autorange': False},
                     'yaxis': {'range': [-3, 3], 'autorange': False},
                     'title': 'Mass Spring Damper',
                     'updatemenus': [{'type': 'buttons',
                                      'buttons': 
  [                 
    {'label': 'Start','method': 'animate',
      'args': [framedata, {'frame': {'duration': 40, 'redraw': False}}]
    },       
    {'label': 'Stop','method': 'animate',
      'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate','transition': {'duration': 0}}]
    }                    
  ]
                                     }]
                    },
          'frames': dummydata
         }

iplot(figure)